# Code comparison between Cloud Data Connector and GCP python API

The following cells show a comparison of how much lines of code are needed to implement Upload and Download routines using Cloud Data Connector and GCP python API. The first cell includes common lines of code between both implementations, these lines will not be considered for the final line count. User should set some variable in order to use the code.

In [ ]:
import os
import tempfile

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "<Path to service account credentials>"
bucket_name = "<bucket_name>"
blob_name = "<blob_name>"
msg = b'Happy days'
project_name = "<project_name>"

f = tempfile.NamedTemporaryFile(delete=False)
f.write(msg)
f.close()
file_path = f.name

The following three cells are Cloud Data Connector code. The basic imports and to obtain the connector only three lines of code are needed.

In [ ]:
from cloud_data_connector.gcp import Connector, Downloader, Uploader

connector = Connector("storage")
storage_client = connector.connect(connection_string=project_name)

The upload routine only needs two lines of code to work.

In [ ]:
uploader = Uploader(storage_client)
uploader.upload_to_bucket(bucket_name, file_path, blob_name)

Finally, to download the uploaded file to GCP storage three lines of code are needed (open and print lines are not counted). In total the complete process needs eight lines to run.

In [ ]:
destination_path = "test1.txt"
downloader = Downloader(storage_client)
downloader.download(bucket_name,blob_name, destination_path)

with open(destination_path, 'r') as file:
    print(file.read())

For GCP python API, three imports plus one line for the storage client declaration are needed.

In [ ]:
from google.cloud import storage
from google.api_core.exceptions import NotFound
from google.cloud.exceptions import GoogleCloudError

storage_client = storage.Client(project=project_name)

To upload the file to a bucket eight lines are required.

In [ ]:
if not os.path.isfile(file_path):
    raise FileNotFoundError("{} not found.".format(data_file))
        
try:
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
except GoogleCloudError as e:
    raise GoogleCloudError('Failed to copy local file {0} to cloud storage file {1}.'.format(file_path, blob_name))

To download the file from storage eight lines are require, giving a total of twenty lines (open and print lines are not counted).

In [ ]:
destination_path = "test2.txt"
try:
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    with open(destination_path, 'wb') as f:
        storage_client.download_blob_to_file(blob, f)
except NotFound as e:
    print("blob not found: {}".format(e))

with open(destination_path, 'r') as file:
    print(file.read())